In [1]:
import json
import os
import string
import re
from nltk.tokenize import sent_tokenize

fpath = "/home/dilyara/Documents/GitHub/alexa-prize-topical-chat-dataset/conversations/"
os.listdir(fpath)

['valid_rare.json',
 'topical_chat_movies',
 'topical_chat_cobot_annotated.tar.gz.md5',
 'topical_chat_movies.tar.gz',
 'topical_chat_sentiment_turn_rating.tar.gz.md5',
 'topical_chat_movies.tar.gz.md5',
 'annotated',
 'valid_freq.json',
 'topical_chat_sentiment_turn_rating',
 'topical_chat_sentiment_turn_rating.tar.gz',
 'test_rare.json',
 'topical_chat_cobot_annotated',
 'topical_chat_cobot_annotated.tar.gz',
 'train.json',
 'test_freq.json']

In [4]:
API_KEY = "MYF6T5vloa7UIfT1LwftY3I33JmzlTaA86lwlVGm"

import requests

headers = {'Content-Type': 'application/json;charset=utf-8', 'x-api-key': API_KEY}
SERVICE_URL = 'https://gpri4649wf.execute-api.us-east-1.amazonaws.com/prod/topic/v1/batchTopics'
utterances=["let's chat about politics", 'I love watching movies.']

result = requests.request(url=SERVICE_URL, headers=headers, data=json.dumps({'utterances': utterances}), method='POST').json()

[r["topicClass"] for r in result["topics"]]

['Politics', 'Movies_TV']

In [18]:
with open(fpath + "train.json", "r") as f:
    data = json.load(f)

In [19]:
string.punctuation


'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [20]:
questions = []
punctuation = '!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~'

for k in data.keys():
    for message in data[k]["content"]:
        sentences = sent_tokenize(message["message"])
        if len(sentences) == 1 and message["message"][-1] == "?":
            punct = re.findall(f"[{punctuation}]", message["message"])
            if punct != ["?"]:
                punct = message["message"].rfind(punct[-2])
            else:
                punct = -1
            quest = message["message"][punct+1:].strip()
            quest = re.sub(f"^hi(\sthere)?[{punctuation}\s]*", "", quest.lower())
            quest = re.sub(f"^hey(\sthere)?[{punctuation}\s]*", "", quest.lower())
            quest = re.sub(f"^hello(\sthere)?[{punctuation}\s]*", "", quest.lower())
            quest = re.sub(f"^yes[{punctuation}\s]*", "", quest.lower())
            quest = re.sub(f"^yeah[{punctuation}\s]*", "", quest.lower())
            quest = re.sub(f"^sure[{punctuation}\s]*", "", quest.lower())
            quest = re.sub(f"^no[{punctuation}\s]*", "", quest.lower())
            if "how are you" in quest.lower():
                quest = ""
            quest = re.sub(f"\t+", " ", quest.lower())
            if len(quest.split()) > 3:
                questions += [quest]
        break

In [21]:
len(questions)

3566

In [22]:
questions = list(set(questions))
len(questions)

2113

In [23]:
batch_size = 100
all_topics = []

for i in range(int(len(questions) / batch_size) + 1):
    utterances = questions[i * batch_size: (i+1)* batch_size]
    try:
        result = requests.request(url=SERVICE_URL, headers=headers, data=json.dumps({'utterances': utterances}), method='POST').json()
    except:
        print(utterances)
        break
    topics = [r["topicClass"] for r in result["topics"]]
    all_topics.extend(topics)

In [24]:
len(all_topics)

2113

In [25]:
import pandas as pd

df = pd.DataFrame({"question": questions, "topic": all_topics})
df.loc[:100, :]

,question,topic
0,did you know that dogs have 12 different blood...,Pets_Animals
1,do you ever watch shows on netflix?,Movies_TV
2,did you know the drive thru was invented in 1975?,Other
3,do you know what the highest scoring football ...,Sports
4,do you like watching baseball?,Sports
...,...,...
96,do you like to laugh?,Phatic
97,do you like star trek?,Phatic
98,what do you think about lane lambert and the p...,Art_Event
99,do you watch basketball?,Sports


In [26]:
df.to_csv("questions_with_topics.csv", index=False)

In [2]:
import pandas as pd

df = pd.read_csv("/home/dilyara/Documents/GitHub/dp-agent-alexa/skills/dummy_skill/questions_with_topics.csv")
df.head()

,question,topic
0,did you know that dogs have 12 different blood...,Pets_Animals
1,do you ever watch shows on netflix?,Movies_TV
2,did you know the drive thru was invented in 1975?,Other
3,do you know what the highest scoring football ...,Sports
4,do you like watching baseball?,Sports


# Cobot Nounphrases

In [5]:
SERVICE_URL = 'http://0.0.0.0:8016/nounphrases'

dialog = {"utterances": [{"text": "Her boobs are in this room tonight"}]}

result = requests.request(url=SERVICE_URL, json={'dialogs': [dialog]}, method='POST').json()
result

[['her boobs', 'this room']]

In [6]:
df_dict = []

for i in range(df.shape[0]):
    df_dict.append({"question": df.loc[i, "question"], "topic": df.loc[i, "topic"]})

In [7]:
from copy import deepcopy

In [8]:
batch_size = 100
all_topics = []

for i in range(int(len(df_dict) / batch_size) + 1):

    dialogs = [deepcopy(dialog) for _ in df_dict[i * batch_size: (i+1)* batch_size]] 

    for dialog, el in zip(dialogs, df_dict[i * batch_size: (i+1)* batch_size]):
        dialog["utterances"][-1]["text"] = el["question"]
    try:
        nounphrases = requests.request(url=SERVICE_URL, json={'dialogs': dialogs}, method='POST').json()
    except:
        print(dialogs)

    for el, np in zip(df_dict[i * batch_size: (i+1)* batch_size], nounphrases):
        el["nounphrases"] = np

In [9]:
with open("data/topical_chat_questions_nouns.json", "w") as f:
    json.dump(df_dict, f, indent=2)

# Clear nounphrases

In [10]:
np_ignore_list = ["'s", 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're",
                  "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself',
                  'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',
                  'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those',
                  'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does',
                  'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of',
                  'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before',
                  'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under',
                  'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any',
                  'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own',
                  'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should',
                  "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn',
                  "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven',
                  "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't",
                  'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't",
                  'wouldn', "wouldn't", "my name", "your name", "wow", "yeah", "yes", "ya", "cool", "okay", "more",
                  "some more", " a lot", "a bit", "another one", "something else", "something", "anything",
                  "someone", "anyone", "play", "mean", "a lot", "a little", "a little bit"]

In [11]:
unique_nps = []

for sample in df_dict:
    for np in sample["nounphrases"]:
        for ignore_np in np_ignore_list:
            np = re.sub("\s\s+", " ", re.sub(r"\b" + ignore_np + r"\b", "", np)).strip()
        if len(np) >= 3:
            unique_nps.append(np)
print(f"Total non-unique nounphrases: {len(unique_nps)}")
unique_nps = list(set(unique_nps))
print(f"Total unique nounphrases: {len(unique_nps)}")

Total non-unique nounphrases: 2601
Total unique nounphrases: 1419


In [12]:
np_to_fact_map = {}

for key in unique_nps:
    np_to_fact_map[key] = []

In [13]:
question_info = {}

total_id = 0

for sample in df_dict:
    question_info[total_id] = sample["question"]
    total_id += 1

In [14]:
for sample_id in question_info.keys():
    for np in np_to_fact_map.keys():
        if np in question_info[sample_id]:
            np_to_fact_map[np] += [sample_id]

In [15]:
bad_nps = []

for key in np_to_fact_map:
    if len(np_to_fact_map[key]) == 0:
        bad_nps.append(key)
        
for np in bad_nps:
    np_to_fact_map.pop(np)

In [16]:
len(np_to_fact_map)

1375

In [17]:
with open(f"data/questions_map.json", "w") as f:
    json.dump(question_info, f, indent=2)

In [18]:
with open(f"data/nounphrases_questions_map.json", "w") as f:
    json.dump(np_to_fact_map, f, indent=2)